In [2]:
import agate

In [3]:
tester = agate.TypeTester(force = {
        'NrdPermit': agate.Text(),
        'PostalCD': agate.Text()
    }, limit=100)
wells = agate.Table.from_csv('wells.csv', column_types=tester)

In [4]:
print(wells)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  X            | Number        |
|  Y            | Number        |
|  WellID       | Number        |
|  RegCD        | Text          |
|  Replacemen   | Number        |
|  Status       | Text          |
|  Useid        | Text          |
|  NrdName      | Text          |
|  NrdID        | Number        |
|  Countyname   | Text          |
|  CountyID     | Number        |
|  Township     | Number        |
|  Range        | Number        |
|  RangeDir     | Text          |
|  Section      | Number        |
|  SubSection   | Text          |
|  FootageNS    | Number        |
|  FootageDir   | Text          |
|  FootageEW    | Number        |
|  FootageD_1   | Text          |
|  NrdPermit    | Text          |
|  Acres        | Number        |
|  SeriesType   | Text          |
|  SeriesEnd    | Date          |
|  PumpRate     | Number        |
|  PColDiam     | Number        |
|  PumpDepth  

In [5]:
oldest = wells.aggregate(agate.Min('Compdate'))

In [6]:
print(oldest)

1895-01-01


In [7]:
newest = wells.aggregate(agate.Max('Compdate'))

In [8]:
print(newest)

2016-02-25


In [9]:
class ActiveWell(agate.Computation):
    """
    Computes whether a well is active in a year
    """
    def __init__(self, column_name_start, column_name_end, year):
        self._column_start = column_name_start
        self._column_end = column_name_end
        self._year = year

    def get_computed_data_type(self, table):
        """
        The return value is a boolean.
        """
        return agate.Boolean()

    def validate(self, table):
        """
        Verify the column is date.
        """
        columnStart = table.columns[self._column_start]

        if not isinstance(columnStart.data_type, agate.Date):
            raise agate.DataTypeError('Can only be applied to date data.')
            
        columnEnd = table.columns[self._column_end]

        if not isinstance(columnEnd.data_type, agate.Date):
            raise agate.DataTypeError('Can only be applied to date data.')

    def run(self, table):
        new_column = []
        
        for row in table.rows:
            start_val = row[self._column_start]
            if start_val is not None:
                start_year = start_val.year
            else:
                start_year = 1700
            end_val = row[self._column_end]
            if end_val is not None:
                end_year = end_val.year
            else:
                end_year = 5000
            if start_year < self._year < end_year:
                status = True
                new_column.append(status)
            else:
                status = False
                new_column.append(status)
        return new_column

In [10]:
active_wells = wells.compute([
    ('2010active', ActiveWell('Compdate', 'DecommDate', 2010))
        ])

In [11]:
checkActive = active_wells.select(['LastName', 'Compdate', 'DecommDate', '2010active'])
checkActive.print_table(10)

|----------------------+------------+------------+-------------|
|  LastName            |   Compdate | DecommDate | 2010active  |
|----------------------+------------+------------+-------------|
|  Binderup            | 2003-01-08 |            |       True  |
|  Letcher             | 2003-05-14 |            |       True  |
|  Keldren Partners    | 1977-06-22 | 1993-05-01 |      False  |
|  Layher              | 1980-11-03 |            |       True  |
|  Elting Farms        | 1955-10-07 | 2007-03-01 |      False  |
|  Carlson             | 1966-01-01 | 2003-09-01 |      False  |
|  Kampfe              | 1937-01-01 |            |       True  |
|  Houser Family Trust | 1969-05-06 | 2001-06-01 |      False  |
|  McConnell           | 2004-05-20 |            |       True  |
|  Schmidt             | 1947-07-01 |            |       True  |
|  ...                 |        ... |        ... |        ...  |
|----------------------+------------+------------+-------------|
